In [1]:
import numpy as np
import pandas as pd
import re
import xlrd
import nltk

In [2]:
#import data
data_path = 'D:\Study\Thesis\Data\Arendeforteckning.csv'
raw_data = pd.read_csv(data_path, sep = ';')

In [4]:
#data preprocessing work
# distinguish with the comma separation and comma within quote
i = 2
for i in range(len(raw_data.index)):
    raw_data['Textbox3'][i] = re.sub(r'"[^"]*"', lambda m: m.group(0).replace(',', ''), raw_data['Textbox3'][i])
    i += 1

#Seperated one column to different attribute, data re-columns based on comma
data = raw_data[raw_data.columns[0]].str.split(',', expand = True)
columns_name = data.iloc[1]
data.columns = columns_name

#Delete first two columns
data = data.drop(data.index[0:2])


In [6]:
#data-cleaning-drop null
data = data.drop(data.index[(data[data.columns[0]] == '')|(data[data.columns[3]] == '')])
data = data.dropna(axis = 1, how = 'all')

In [7]:
#Data re-columns
Titles =       ['Contact_type',
                'Complaint',
                'Clinic',
                'Content',
                'Case_num',
                'Contactor',
                'Problem_Cat1',
                'SubCat1',
                'Case_status',
                'Problem_Cat2',
                'SubCat2',
                'Problem_Cat3',
                'Subcat3',
                'Flag']

data.columns = Titles

data['Content'] = data['Content'].str.replace("\"","")

#Re-index
data = data.reset_index()
data = data.drop(['index'],axis = 1)

In [12]:
#Select Candidate as the first iteration of sentiment analysis
Candidate = data[['Content']].copy()

#transpose all big letter to small letter
Candidate['Content']= Candidate['Content'].str.lower()

#Removing Punctuation
Candidate['Content']= Candidate['Content'].str.replace('[^\w\s]','')

#Removal of commonly occurring words
stop = nltk.corpus.stopwords.words('swedish')
Candidate['remove']= Candidate['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [13]:
#Common word removal
freq = pd.Series(' '.join(Candidate['remove']).split()).value_counts()[:35]
# DO not remove ['läkaren','läkare']
not_remove=['läkaren','läkare']
freq=freq.drop(labels=not_remove)
#freq
freq = list(freq.index)
Candidate['remove'] = Candidate['remove'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [14]:
#Rare words removal
freq_rare = pd.Series(' '.join(Candidate['remove']).split()).value_counts()
freq_rare = freq_rare[freq_rare == 1]

freq_rare = list(freq_rare.index)
Candidate['remove'] = Candidate['remove'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_rare))



In [16]:
#Tokensize remove sentences
Candidate['tokenize_sents'] = Candidate.apply(lambda row: nltk.word_tokenize(row['remove']), axis=1)

In [15]:
Candidate.head()

,Content,remove,tokenize_sents
0,en kvinna föll och ådrog sig en skada i ena ax...,föll ådrog skada ena axeln fortfarande stora s...,"[föll, ådrog, skada, ena, axeln, fortfarande, ..."
1,en kvinna hade genomgått en operation och vård...,genomgått vårdades tillsammans andra patienter...,"[genomgått, vårdades, tillsammans, andra, pati..."
2,en man med symtom från mag tarmkanalen sökte a...,symtom akutmottagning sjukhuset gånger rätt di...,"[symtom, akutmottagning, sjukhuset, gånger, rä..."
3,en kvinna med bröstcancer har genomgått strålb...,genomgått ena bröstet behandlingen försökt kom...,"[genomgått, ena, bröstet, behandlingen, försök..."
4,en kvinna hade problem med trötthet och stort ...,problem stort genomgick undersökningar svar vi...,"[problem, stort, genomgick, undersökningar, sv..."


In [17]:
import xlsxwriter
writer = pd.ExcelWriter('CleanData1.xlsx')
Candidate.to_excel(writer,'sheet1')
writer.save()